In [ ]:

!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# LangChain multi-doc retriever with ChromaDB

In [ ]:
from langchain.vectorstores import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
# data source, https://themarketswatch.com/financial/top-stocks-to-buy/?gclid=CjwKCAjwyqWkBhBMEiwAp2yUFpSLJZSVwVHJR3co5yVHVLeBPqNNcU3VMrFJjniP3jtuupP6kHzDYxoC8dkQAvD_BwE

!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip
!unzip stock_market_june_2023.zip -d /content/


--2023-06-15 03:27:21--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2023-06-15 03:27:21--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2023-06-15 03:27:22 (25.5 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]


In [ ]:

loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)
#documents = loader.load()#[:5]  # Only use the first 5 files
documents = loader.load()

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

49

In [ ]:
texts[0]

Document(page_content='The stock market had a strong month in May, especially for those who read our previous monthly stock recommendation. At The Markets Watch, our journalists and analysts have compiled a list of monthly stock opportunities for buy-and-hold investors and traders looking to buy great companies at an even better price.\nLooking at some of our stock suggestions from the previous month, the evidence of our team’s expertise becomes compelling. In May:', metadata={'source': 'stock_market_june_2023/stock_market.txt'})

In [ ]:
!pip install auto-gptq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# TheBloke/Nous-Hermes-13B-GPTQ

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

# HuggingFace Embeddings

In [ ]:

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

# Moving model to GPU if available
#model = model.to(device)

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length= 2048,
#     temperature=0.7,
#     top_p=0.95,
#     repetition_penalty=1.15)
# # #device=0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU
# # )



pipe = pipeline(
    model=model, tokenizer= tokenizer,
    return_full_text=True,  # langchain expects the full text
    task= 'text-generation',
    device= "cuda:0",
    temperature=0.7,
    top_p=0.95,  #
    max_new_tokens=1028,  # mex number of tokens to generate in the output
    repetition_penalty=1.15  # without this output begins repeating
)


llm = HuggingFacePipeline(pipeline= pipe)

model_name = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)



The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

# create the DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

embedding = hf

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    unique_sources = set(source.metadata['source'] for source in llm_response["source_documents"])
    for source in unique_sources:
        print(source)


In [ ]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 According to the passage, Microsoft has invested $1 billion in OpenAI.


Sources:
stock_market_june_2023/Microsoft Corporation.txt


In [ ]:
from tensorflow import convert_to_tensor

In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The companies mentioned as potential growth stocks are Arbor Metals Corp., Tesla Inc., Kiplin Metals Inc., Microsoft, NVIDIA, Shopify, Intuitive Surgical, MercadoLibre, and Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience.


Sources:
stock_market_june_2023/Microsoft Corporation.txt
stock_market_june_2023/stock_market.txt


In [ ]:
query = "how many companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 There are 8 companies that have been highlighted for their potential stock growth.


Sources:
stock_market_june_2023/stock_market.txt


In [ ]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20%, selling its logistics business to Flexport, and reducing overhead costs. These changes were made to improve efficiency and profitability in the long run.


Sources:
stock_market_june_2023/Shopify.txt


In [ ]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 NVIDIA's dominance in the GPU market provides them with a strong foothold in the AI industry due to their ability to provide high-performance hardware that is essential for training AI models. This advantage allows them to capture a significant portion of the market share and establish themselves as a key player in the rapidly growing AI space.


Sources:
stock_market_june_2023/NVIDIA Corporation.txt


In [ ]:
query = "What are the top five companies and how much did their stock value increase?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 The top five companies with the highest percentage increase in stock value are Palantir (74%), Nvidia (34%), Crowdstrike Holdings (30%), Shopify (19%), and Our Insights (57%).


Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/NVIDIA Corporation.txt
